### 3대 지랄견 유기율 데이터 분석
- 가설: 3대 지랄견으로 불리는 코카 스패니얼, 비글, 슈나우저는 유기율이 높을까?

- 사용 라이브러리 로드

In [1]:
from database.Database import Database
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import folium
import json

In [2]:
db = Database(
    host = "database-2.c3iym8yog7ht.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "pethub",
    password = "addinedu5",
    database = "pethub"
)

In [5]:
sql =   """
        SELECT l.region, p.breed, r.count
        FROM region_pet_regist r
        LEFT JOIN region_list l ON r.region_id = l.id
        LEFT JOIN pet_type p ON r.pet_type_id = p.id;
        """
result = db.selectData(sql)

regist_df = pd.DataFrame(result)
regist_df.columns = ["region", "breed", "pet_count"]
regist_df.head()

,region,breed,pet_count
0,종로구,골든 리트리버,156
1,종로구,그레이 하운드,5
2,종로구,달마시안,3
3,종로구,도베르만,21
4,종로구,라브라도 리트리버,136
